<a href="https://colab.research.google.com/github/youngyoung-0/colab-notebook/blob/main/python_ML_study/BERT_NEWS_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install datasets
!pip install transformers

In [35]:
from datasets import load_dataset

import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

# BERT
import torch
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertConfig
from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# for padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 전처리 및 평가 지표
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

In [36]:
df = pd.read_csv('finance_data.csv')
df.head()

,labels,sentence,kor_sentence
0,neutral,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [37]:
df['labels'].value_counts()

,count
labels,
neutral,2879
positive,1363
negative,604


In [38]:
df['labels'] = df['labels'].replace(['neutral', 'positive', 'negative'], [0, 1, 2])
df

/tmp/ipython-input-3113801322.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['labels'] = df['labels'].replace(['neutral', 'positive', 'negative'], [0, 1, 2])


,labels,sentence,kor_sentence
0,0,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,0,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,2,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,1,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,1,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."
...,...,...,...
4841,2,LONDON MarketWatch -- Share prices ended lower...,런던 마켓워치 -- 은행주의 반등이 FTSE 100지수의 약세를 상쇄하지 못하면서 ...
4842,0,Rinkuskiai's beer sales fell by 6.5 per cent t...,린쿠스키아의 맥주 판매량은 416만 리터로 6.5% 감소했으며 카우노 알루스의 맥주...
4843,2,Operating profit fell to EUR 35.4 mn from EUR ...,"영업이익은 2007년 68.8 mn에서 35.4 mn으로 떨어졌으며, 선박 판매 이..."
4844,2,Net sales of the Paper segment decreased to EU...,페이퍼 부문 순매출은 2008년 2분기 241.1 mn에서 2009년 2분기 221...


In [39]:
df.to_csv('new_finance_data.csv', index=False, encoding='utf-8')

In [40]:
all_data=load_dataset('csv', data_files={"train": 'new_finance_data.csv'})
all_data

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'sentence', 'kor_sentence'],
        num_rows: 4846
    })
})

In [41]:
cs = all_data['train'].train_test_split(0.2, seed=777)
train_cs = cs['train']
test_cs = cs['test']
test_cs

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 970
})

In [42]:
# 훈련데이터를 다시 8:2로 분리 후 훈련 데이터와 검증 데이터로 저장
cs = train_cs.train_test_split(0.2, seed=777)
train_cs = cs['train']
valid_cs = cs['test']

In [43]:
# 데이터 전처리
# 훈련데이터, 검증 데이터, 테스트 데이터 대해서 [CLS], [SEP]구조 생성
train_sentences = list(map(lambda x: '[CLS]' + str(x) + '[SEP]', train_cs['kor_sentence']))
validation_sentences = list(map(lambda x: '[CLS]' + str(x) + '[SEP]', valid_cs['kor_sentence']))
test_sentences = list(map(lambda x: '[CLS]' + str(x) + '[SEP]', test_cs['kor_sentence']))

train_sentences[:10]

['[CLS]이 새로운 주식은 2006 회계연도에 배당받을 수 있는 권리를 보유자들에게 부여한다.[SEP]',
 '[CLS]알마 미디어 코퍼레이션 사업 ID 1944757-4의 주식 자본금은 44,767,513.80유로이며 74,612,523주로 나뉜다.[SEP]',
 '[CLS]사업 부문에는 알스트롬의 지속가능성 보고서도 포함돼 있다.[SEP]',
 '[CLS]2005년 1/4분기 EUR 5.2 mn에 비해 세전 이익은 총 4.9 mn입니다.[SEP]',
 '[CLS]핀란드 헬싱키에 본사를 둔 레민카이넨 그룹은 토목 공학, 건축 계약, 기술 건축 서비스, 건축 자재 산업 등 건설 산업의 모든 부문에서 운영되고 있다.[SEP]',
 '[CLS]핀란드 에스포에 본사를 둔 코네는 세계 최고의 엘리베이터 및 에스컬레이터 회사 중 하나이다.[SEP]',
 '[CLS]그 회사는 2005년에 배당을 하지 않았다.[SEP]',
 '[CLS]이에 상응하는 주식증자 1,012,945.50유로가 오늘 무역대장에 등록되었습니다.[SEP]',
 '[CLS]분할된 활동은 2009년에 1억 4,510만 유로의 순매출과 890만 유로의 영업이익을 기록했다.[SEP]',
 '[CLS]바이오티 치료법 주식회사의 자기자본은 총 9021만1860주를 구성하며 이 주식에 대한 의결권 보유 수는 9021만1860주에 달한다.[SEP]']

In [44]:
train_labels = train_cs['labels']
validation_labels = valid_cs['labels']
test_labels = test_cs['labels']

In [45]:
# BERT 토크나이저를 이용한 전처리
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [46]:
# 최대 길이
max_len = 128

def data_to_tensor(sentences, lables, max_len):
  tokenizer_texts = [tokenizer.tokenize(sent) for sent in sentences]
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenizer_texts]

  # pad_sequences
  input_ids = pad_sequences(input_ids, maxlen = max_len, dtype = 'long', truncating = 'post', padding='post')

  # 실제 토큰 위치에는 1, 패딩 토큰 위치에는 0을 넣은 리스트인 어텐션 마스크 제작
  # 정수 인코딩 결과가 [231, 52, 45, 0, 0, 0]이 있다면 231, 52, 45는 실제 토큰이고 0은 패딩 토큰이므로
  # 어텐션 마스크는 [1, 1, 1, 0, 0, 0]
  attention_masks = []

  for seq in input_ids:
    seq_mask = [float(i > 0) for i in seq]
    attention_masks.append(seq_mask)

  tensor_inputs = torch.tensor(input_ids)
  tensor_labels = torch.tensor(lables)
  tensor_masks = torch.tensor(attention_masks)
  return tensor_inputs, tensor_labels, tensor_masks

In [47]:
train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels, max_len)
validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_sentences, validation_labels, max_len)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentences, test_labels, max_len)


In [48]:
# 데이터의 배치화, 배치학습 위한 데이터로드
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = batch_size)


validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler = validation_sampler, batch_size = batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler= test_sampler, batch_size= batch_size)

In [49]:
# GPU 세팅
if torch.cuda.is_available():
  device = torch.device('cuda')
  print(f"There are {torch.cuda.device_count()} GPU(s) available")
  print('We will use the GPU : ', torch.cuda.get_device_name(0))

else:
  device = torch.device('cpu')
  print('no GPU available, using the CPU instead')

There are 1 GPU(s) available
We will use the GPU :  Tesla T4


In [50]:
# 모델 로드
num_labels = 3
model = BertForSequenceClassification.from_pretrained('klue/bert-base', num_labels= num_labels)
model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [51]:
# 모델 학습
epochs = 3

# 옵티마니저 선택
optimizer = AdamW(model.parameters(), lr = 2e-5)

def metrics ( perdictions, labels):
  # predictions : 모델이 예측한 결과값들의 리스트 또는 배열
  # labels : 실제 정답 레이블들의 리스트 또는 배열

  # 예측값과 실제 레이블을 별도의 변수에 할당
  y_pred = perdictions
  y_true = labels

  # Accuracy
  accuracy = accuracy_score(y_true, y_pred)

  # Macro_averaged F1 Score (클래스별 점수를 단순 평균)
  f1_macro_average = f1_score(y_true= y_true, y_pred = y_pred, average = 'macro', zero_division = 0)

  # micro-averaged F1 (전체 샘플의 개수로 점수 계산)
  f1_micro_average = f1_score(y_true= y_true, y_pred = y_pred, average = 'micro', zero_division = 0)

  # weight-average F1 score (각 클래스의 데이터의 개수(weight)곱해서 )
  f1_weight_average = f1_score(y_true= y_true, y_pred = y_pred, average = 'weighted', zero_division = 0)

  metrics = {
      # 정확도 : 전체 예측 중에서 올바르게 예측한 비율을 나타냅니다.
      'accuracy' : accuracy,
      # 전체 데이터에 대해 단일 F1 점수를 계산합니다. 클래스 불균형이 심한 경우에 적합
      'f1_macro' : f1_macro_average,
      # 전체 데이터에 대해 단일 F1 점수를 계산. 클래스 불균형이 심한 경우에 적합
      'f1_micro' : f1_micro_average,
      # 클래스의 샘풀 수를 가중치로 곱한 후 평균을 구합니다. 클래스 불균형을 고려
      'f1_weight' : f1_weight_average
  }

  return metrics

In [52]:
def train_epoch(model, train_dataloader, optimizer, device):
  """
  한 에포크 동안 모델을 학습시키는 함수

  parameters :
  model (torch.nn.Moudle) : 학습시킬 모델 객체
  train_dataloader (torch.utils.data.DataLoader) : 학습 데이터셋의 DataLoader
  optimizer(torch.optimizer) : 최적화 알고리즘을 구현하는 객체
  device (troch.device) : 학습에 사용할 장치 (CPU or cuda)

  """

  total_train_loss = 0 # 학습 손실을 누적할 변수 초기화
  model.train() # 모델을 학습 모드로 전환

  # 학습 데이터로더를 순회하며 배치 단위로 학습
  for step, batch in tqdm(enumerate(train_dataloader), desc='Training Batch'):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch # 배치에서 입력 ID, 마스크, 라벨 추출

    # 모델에 배치를 전달하여 손실값 계산
    outputs = model(b_input_ids, token_type_ids = None, attention_mask = b_input_mask, labels = b_labels)
    # 손실값 추출
    loss = outputs.loss

    optimizer.zero_grad() # 기울기(gradient) 초기화
    loss.backward() # 역전파를 통해 기우기(gradient) 계산
    optimizer.step() # 가중치 업데이트

  avg_train_loss = total_train_loss / len(train_dataloader) # 평균 학습 손실
  return avg_train_loss # 평균 학습 손실 반환




In [53]:
# 학습 데이터로더를 순회하면서 배치(batch) 단위로 학습 진행
def evaluate(model, validation_dataloader, device):
  """
  모델을 사용하여 검증 데이터셋에 대한 평가를 수행하는 함수

  Parameters :
  model ( torch.nn.Module ) : 평가할 모델 객체
  validation_dataloader (torch.utils.data.DataLoader) : 검증 데이터넷의 DataLoader.
  device ( troch.device ) : 평가에 사용할 장치

  returns:
  float : 평균 검증 손실값,
  dict : 다양한 평가 지표 (metrics)에 대한 값들을 담은 사전.
  """

  model.eval() # 모델을 평가 모드로 설정

  total_eval_loss = 0 # 검증 손실을 누적할 변수 초기화
  predictions, true_labels = [], [] # 예측값과 실제 라벨값을 지정할 리스트 초기화

  # 검증 데이터로더를 순회하며 배치 단위로 평가
  for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch) # 배치 데이터를 디바이스로 이동
    b_input_ids, b_input_mask, b_labels = batch # 배치에서 입력 ID, 마스크, 라벨 추출

    with torch.no_grad():
      # 모델에 배치를 전달하여 손실값 계산
      outputs = model(b_input_ids,token_type_ids = None, attention_mask = b_input_mask, labels = b_labels)

    # 모델 출력에서 손실값 추출
    if outputs.loss is not None :
      loss = outputs.loss
      total_eval_loss += loss.item() # 총 손실에 더함

    logits = outputs.logits.detach().cpu().numpy() # 모델 예측값(로짓)을 numpy 배열로 변환
    label_ids = b_labels.to('cpu').numpy()# 실제 라벨값을 Numpy 배열로 변환

    # 3개의 값 중 가장 큰 값을 예측한 인덱스로 결정 ()
    predictions.extend(np.argmax(logits, axis = 1).flatten()) # 예측된 클래스를 리스트에 추가
    true_labels.extend(label_ids.flatten()) # 실제 레이블 값을 리스트에 추가

  eval_metrics = metrics(predictions, true_labels)
  return total_eval_loss / len(validation_dataloader), eval_metrics

In [56]:
 # 최소 검증 손실 초기화
min_val_loss = float('inf')

# 메인 학습 & 평가 루트
for epoch_i in range(0, epochs):
  print(' ==== Epoch {:} / {:} ===='.format(epoch_i + 1, epochs))

  # 학습 단계
  train_epoch(model, train_dataloader, optimizer, device)

  print('\n Runnin Validation...')

  # 검증 단계
  avg_val_loss, eval_metrics = evaluate(model, validation_dataloader, device)

  print('Validation Loss : {0:.2f}'.format(avg_val_loss))
  print('Validation Accuracy : {0:.2f}'.format(eval_metrics['accuracy']))
  print('Validation F1 Score (Macro) : {0:.2f}'.format(eval_metrics['f1_macro']))
  print('Validation F1 Score (Micro) : {0:.2f}'.format(eval_metrics['f1_micro']))
  print('Validation F1 Score (Weight) : {0:.2f}'.format(eval_metrics['f1_weight']))

  # 검증 손실이 현재까지의 최소값보다 작은 경우 체크포인트 저장
  if avg_val_loss < min_val_loss:
    print(f'Validation loss decreased ({min_val_loss :.2f} --> {avg_val_loss:.2f}). Saving model ...')
    # 베스트 모델 저장
    torch.save(model.state_dict(), 'model_checkpoint.pt')
    # 최소 검증 손실 업데이트
    min_val_loss = avg_val_loss

 ==== Epoch 1 / 3 ====


Training Batch: 97it [01:00,  1.61it/s]



 Runnin Validation...
Validation Loss : 0.39
Validation Accuracy : 0.84
Validation F1 Score (Macro) : 0.81
Validation F1 Score (Micro) : 0.84
Validation F1 Score (Weight) : 0.84
Validation loss decreased (inf --> 0.39). Saving model ...
 ==== Epoch 2 / 3 ====


Training Batch: 97it [00:59,  1.63it/s]



 Runnin Validation...
Validation Loss : 0.43
Validation Accuracy : 0.84
Validation F1 Score (Macro) : 0.81
Validation F1 Score (Micro) : 0.84
Validation F1 Score (Weight) : 0.84
 ==== Epoch 3 / 3 ====


Training Batch: 97it [00:59,  1.63it/s]



 Runnin Validation...
Validation Loss : 0.46
Validation Accuracy : 0.83
Validation F1 Score (Macro) : 0.80
Validation F1 Score (Micro) : 0.83
Validation F1 Score (Weight) : 0.83


각 epoch별로 trian_epoch() 함수를 호출하여 한 에포크 동안 모델을 학습시킨다. 한 에포크의 학습이 끝나면 evaluate() 함수를 호출하여 검증 데이터셋에 대한 모델의 성능을 평가한다.

이후, if avg_val_loss < min_val_loss : 코드에서는, 현재 에포크의 검증 데이터에 대한 오차가 지금까지의 최소 검증 손실보다 작은지 확인한다. (가장 좋은 성능을 보인 모델을 다로 저장)

모든 에포크에 대해 반복하낟. 각 에포크마다 모델은 학습 데이터로 업데이트되고, 검증 데이터로 평가된다. 에포크가 진행될수록 모델의 성능이 향상되기를 기대한다.

In [57]:
# 베스트 모델 로드 및 평가
model.load_state_dict(torch.load('model_checkpoint.pt'))


avg_val_loss, eval_metrics = evaluate(model, test_dataloader, device)

print(" Test Loss : {0:.2f}".format(avg_val_loss))
print(" Test Accuracy : {0:.2f}".format(eval_metrics['accuracy']))
print(" Test F1 Score (Macro) : {0:.2f}".format(eval_metrics['f1_macro']))
print(" Test F1 Score (Micro) : {0:.2f}".format(eval_metrics['f1_micro']))
print(" Test F1 Score (Weight) : {0:.2f}".format(eval_metrics['f1_weight']))

 Test Loss : 0.37
 Test Accuracy : 0.84
 Test F1 Score (Macro) : 0.83
 Test F1 Score (Micro) : 0.84
 Test F1 Score (Weight) : 0.84


In [60]:
from transformers import pipeline

pipe = pipeline('text-classification', model = model.cuda(), tokenizer = tokenizer, device = 0, max_length = 512, return_all_scores = True, function_to_apply='softmax')
result = pipe('SK하이닉스가 매출이 급성장하였습니다.')
print(result)

Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[[{'label': 'LABEL_0', 'score': 0.05365106090903282}, {'label': 'LABEL_1', 'score': 0.9357607960700989}, {'label': 'LABEL_2', 'score': 0.010588168166577816}]]


In [63]:
# return_all_scores 제거
pipe = pipeline('text-classification', model = model.cuda(), tokenizer = tokenizer, device = 0, max_length = 512, function_to_apply = 'softmax')
result = pipe('SK하이닉스가 매출이 급성장하였습니다.')
print(result)

Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.9357607960700989}]


In [66]:
label_dict = {'LABEL_0' : '중립', 'LABEL_1': '긍정', 'LABEL_2' : '부정'}
def prediction(text):
  result = pipe(text)

  return [label_dict[result[0]['label']]]

In [67]:
prediction('네이버 매출이 급성장하였습니다.')

['긍정']